In [1]:
%load_ext watermark
%watermark -v -m

Python implementation: CPython
Python version       : 3.7.4
IPython version      : 7.16.1

Compiler    : Clang 4.0.1 (tags/RELEASE_401/final)
OS          : Darwin
Release     : 17.7.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit



In [2]:
import pandas as pd
import json
import os
import shutil
import sys

import ray
import ray.rllib.agents.ppo as ppo

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
checkpoint_root = "tmp/ppo/cart"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)

ray_results = f'{os.getenv("HOME")}/ray_results/'
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

In [4]:
info = ray.init(ignore_reinit_error=True)

File descriptor limit 256 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'
2021-04-28 15:20:48,110	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


In [5]:
SELECT_ENV = "CartPole-v1"                      # Specifies the OpenAI Gym environment for Cart Pole
N_ITER = 10                                     # Number of training runs.

config = ppo.DEFAULT_CONFIG.copy()              # PPO's default configuration. See the next code cell.
config["log_level"] = "WARN"                    # Suppress too many messages, but try "INFO" to see what can be printed.

# Other settings we might adjust:
config["num_workers"] = 1                       # Use > 1 for using more CPU cores, including over a cluster
config["num_sgd_iter"] = 10                     # Number of SGD (stochastic gradient descent) iterations per training minibatch.
                                                # I.e., for each minibatch of data, do this many passes over it to train. 
config["sgd_minibatch_size"] = 250              # The amount of data records per minibatch
config["model"]["fcnet_hiddens"] = [100, 50]    #
config["num_cpus_per_worker"] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

In [6]:
agent = ppo.PPOTrainer(config, env=SELECT_ENV)

results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {
        "n": n,
        "episode_reward_min": result["episode_reward_min"],
        "episode_reward_mean": result["episode_reward_mean"], 
        "episode_reward_max": result["episode_reward_max"],  
        "episode_len_mean": result["episode_len_mean"],
    }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    
    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

2021-04-28 15:20:50,842	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-04-28 15:20:50,843	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=5591) WARNING:tensorflow:From /opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=5591) Instructions for updating:
(pid=5591) non-resource variables are not supported in the long term
2021-04-28 15:20:52,961	WARNING worker.py:1107 -- Failed to unpickle actor class 'RolloutWorker' for actor ID a67dc375e60ddd1a23bd3bb901000000. Traceback:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/function_manager.py", line 496, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
  File "

RayTaskError(NameError): [36mray::RolloutWorker.foreach_policy()[39m (pid=5591, ip=192.168.1.65)
  File "python/ray/_raylet.pyx", line 439, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 442, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 459, in ray._raylet.execute_task
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/serialization.py", line 245, in deserialize_objects
    self._deserialize_object(data, metadata, object_ref))
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/serialization.py", line 192, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata_fields)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/serialization.py", line 170, in _deserialize_msgpack_data
    python_objects = self._deserialize_pickle5_data(pickle5_data)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/serialization.py", line 160, in _deserialize_pickle5_data
    obj = pickle.loads(in_band)
  File "/opt/anaconda3/lib/

In [ ]:
df = pd.DataFrame(data=episode_data)
df

In [ ]:
df.plot(x="n", y=["episode_reward_mean", "episode_reward_min", "episode_reward_max"], secondary_y=True)

In [ ]:
ray.shutdown()